<a href="https://colab.research.google.com/github/Prabha14039/landslide-prediction/blob/main/landslide_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv('Book2new.csv')

# Define categorical features
categorical_features = ['State', 'District', 'Material Type', 'Movement Type', 'Failure Plane', 'Activity', 'Failure Mechanism', 'Geomorphology', 'Hydrology', 'Risk', 'Triggering Factor', 'Geoscientific Causes']

# Handling NaN values and frequency encoding for each feature
for feature in categorical_features:
        dataset[feature].fillna('Unknown', inplace=True)  # Fill NaN with 'Unknown'
        freq_encoding = dataset[feature].value_counts(normalize=True)  # Frequency encoding
        dataset[f'{feature}_Frequency_Encoded'] = dataset[feature].map(freq_encoding)  # Create new encoded column

dataset['Height'].fillna('Unknown', inplace=True)

# Print the updated dataset
print(dataset.isnull().sum())

State                                     0
District                                  0
Longitude                                 0
Latitude                                  0
Length                                    0
Width                                     0
Height                                    0
Material Type                             0
Movement Type                             0
Failure Plane                             0
Activity                                  0
Failure Mechanism                         0
Geomorphology                             0
Hydrology                                 0
Risk                                      0
Triggering Factor                         0
Geoscientific Causes                      0
LocationCluster                           0
State_Frequency_Encoded                   0
District_Frequency_Encoded                0
Material Type_Frequency_Encoded           0
Movement Type_Frequency_Encoded           0
Failure Plane_Frequency_Encoded 

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Assuming 'dataset' is your DataFrame with categorical and numerical features
# Let's say 'dataset' already contains 'Category_Frequency_Encoded' and other categorical features in object dtype
features_to_keep = ['Longitude', 'Latitude', 'Length', 'Width',
                    'Material Type_Frequency_Encoded', 'Failure Plane_Frequency_Encoded',
                    'Geomorphology_Frequency_Encoded']

# Select only the columns you want to keep
dataset_filtered = dataset[features_to_keep]
# Select numerical columns
numerical_columns = dataset_filtered.select_dtypes(include=['int', 'float']).columns.tolist()

# Prepare X_numerical with numerical columns
X_numerical = dataset[numerical_columns]

X=X_numerical
y=dataset['Risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Select features based on importance
sfm = SelectFromModel(rf,threshold='median',prefit=True)
X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

# Retrieve the selected feature indices
selected_indices = sfm.get_support(indices=True)

# Get the names of selected features
selected_feature_names = X_train.columns[selected_indices]
print("Selected Feature Names:")
print(selected_feature_names)

# Filter the dataset to keep only selected features
dataset_selected = dataset[selected_feature_names]

pca_selected = PCA(n_components=0.9)  # Adjust as per explained variance ratio desired
X_train_pca_selected = pca_selected.fit_transform(X_train_selected)
X_test_pca_selected = pca_selected.transform(X_test_selected)

# Print the new PCA components
print("PCA Components for Selected Features:")
print(pca_selected.components_)
print("Explained Variance Ratio:")
print(pca_selected.explained_variance_ratio_)





Selected Feature Names:
Index(['Longitude', 'Latitude', 'Length', 'Width'], dtype='object')
PCA Components for Selected Features:
[[-0.00109291  0.00137233  0.95713905  0.28962347]
 [-0.01968952  0.01645245 -0.28957077  0.95681263]]
Explained Variance Ratio:
[0.87244037 0.12720139]


In [10]:
X_train


,Longitude,Latitude,Length,Width,Material Type_Frequency_Encoded,Failure Plane_Frequency_Encoded,Geomorphology_Frequency_Encoded
18,75.975920,33.325900,70,40,0.117470,0.424699,0.125000
516,79.650250,30.093000,40,25,0.322289,0.427711,0.194277
550,79.522528,30.084222,15,5,0.322289,0.427711,0.194277
463,79.610048,30.100209,25,15,0.322289,0.427711,0.143072
61,75.764070,33.381160,60,40,0.087349,0.424699,0.125000
...,...,...,...,...,...,...,...
71,75.806270,33.358780,40,60,0.117470,0.424699,0.125000
106,75.893320,33.344010,65,85,0.117470,0.424699,0.125000
270,74.129959,34.379627,100,300,0.245482,0.015060,0.037651
435,79.609917,30.041083,90,40,0.245482,0.427711,0.194277


In [12]:
from pycaret.classification import *
from sklearn.model_selection import train_test_split
import pandas as pd

# X_train, X_test, y_train, y_test = train_test_split(X_numerical_pca_df, dataset['Risk'], test_size=0.2, random_state=42)

clf1 = setup(data=pd.concat([X_train, y_train], axis=1), target='Risk', session_id=42)

# Compare models and tune hyperparameters (optional steps)
best_model = compare_models()

# Evaluate the Best Model on the test set
evaluate_model(best_model)

# Make predictions on the test set
predictions = predict_model(best_model, data=X_test)


,Description,Value
0,Session id,42
1,Target,Risk
2,Target type,Multiclass
3,Target mapping,"High: 0, Low: 1, Low or absent: 2, Moderate: 3"
4,Original data shape,"(431, 8)"
5,Transformed data shape,"(431, 8)"
6,Transformed train set shape,"(301, 8)"
7,Transformed test set shape,"(130, 8)"
8,Numeric features,7
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.7209,0.0000,0.7209,0.7166,0.7062,0.4938,0.5043,0.0430
et,Extra Trees Classifier,0.7177,0.8451,0.7177,0.7063,0.7043,0.4746,0.4828,0.2130
xgboost,Extreme Gradient Boosting,0.7145,0.8365,0.7145,0.7118,0.6982,0.4693,0.4812,0.1380
rf,Random Forest Classifier,0.7110,0.8579,0.7110,0.7199,0.6989,0.4613,0.4757,0.3020
lightgbm,Light Gradient Boosting Machine,0.6945,0.8222,0.6945,0.6875,0.6785,0.4276,0.4382,0.8580
gbc,Gradient Boosting Classifier,0.6911,0.0000,0.6911,0.7057,0.6858,0.4411,0.4507,0.4880
dt,Decision Tree Classifier,0.6777,0.7175,0.6777,0.6898,0.6713,0.4271,0.4358,0.0480
ridge,Ridge Classifier,0.6613,0.0000,0.6613,0.5944,0.6116,0.2884,0.3137,0.0440
nb,Naive Bayes,0.6081,0.7667,0.6081,0.6406,0.5966,0.3549,0.3805,0.0440
dummy,Dummy Classifier,0.6046,0.5000,0.6046,0.3657,0.4557,0.0000,0.0000,0.0390


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [3]:
from pycaret.regression import *

clf1 = setup(data=pd.concat([X_train, y_train], axis=1), target='Risk', session_id=42)

# Compare Regression Models
best_model = compare_models()

# Evaluate the Best Model on the test set
evaluate_model(best_model)

# Make predictions on the test set
predictions = predict_model(best_model, data=X_test)

,Description,Value
0,Session id,42
1,Target,Risk
2,Target type,Regression
3,Original data shape,"(398, 8)"
4,Transformed data shape,"(398, 8)"
5,Transformed train set shape,"(278, 8)"
6,Transformed test set shape,"(120, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

AttributeError: 'list' object has no attribute 'predict'

In [11]:
# from sklearn.impute import SimpleImputer

# # Impute categorical columns with the most frequent value
# categorical_imputer = SimpleImputer(strategy='most_frequent')
# dataset[categorical_columns] = categorical_imputer.fit_transform(dataset[categorical_features])

# Check for remaining NaN values after imputation
# print(dataset.isnull().sum())
print(X_numerical.isnull().sum())

Longitude                                 0
Latitude                                  0
Length                                    0
Width                                     0
Height                                    5
LocationCluster                           0
State_Frequency_Encoded                   0
District_Frequency_Encoded                0
Material Type_Frequency_Encoded           0
Movement Type_Frequency_Encoded           0
Failure Plane_Frequency_Encoded           0
Activity_Frequency_Encoded                0
Failure Mechanism_Frequency_Encoded       0
Geomorphology_Frequency_Encoded           0
Hydrology_Frequency_Encoded               0
Risk_Frequency_Encoded                    0
Triggering Factor_Frequency_Encoded       0
Geoscientific Causes_Frequency_Encoded    0
dtype: int64


In [7]:
# # Step 6: Dimensionality reduction using PCA
# pca = PCA(n_components=0.95)  # Retain 95% of variance
# X_train_pca = pca.fit_transform(X_train_selected)
# X_test_pca = pca.transform(X_test_selected)

# # Step 7: Assess feature importance or correlation analysis if needed
# # Example: Printing feature importances from the Random Forest model
# feature_importances = pd.Series(rf.feature_importances_, index=X.columns)
# print("Feature Importances:")
# print(feature_importances)

# # Example: Print PCA components and explained variance ratio
# print("PCA Components:")
# print(pca.components_)
# print("Explained Variance Ratio:")
# print(pca.explained_variance_ratio_)


# Apply PCA on numerical features without scaling
# pca = PCA(n_components=0.95)  # Adjust n_components based on explained variance ratio desired
# X_numerical_pca = pca.fit_transform(X_numerical)


# X=X_numerical_pca
# y=dataset['Risk']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Example: Training a RandomForestClassifier
# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)
# accuracy = clf.score(X_test, y_test)
# print(f"Accuracy: {accuracy}")

Longitude                                 0
Latitude                                  0
Length                                    0
Width                                     0
Height                                    5
LocationCluster                           0
State_Frequency_Encoded                   0
District_Frequency_Encoded                0
Material Type_Frequency_Encoded           0
Movement Type_Frequency_Encoded           0
Failure Plane_Frequency_Encoded           0
Activity_Frequency_Encoded                0
Failure Mechanism_Frequency_Encoded       0
Geomorphology_Frequency_Encoded           0
Hydrology_Frequency_Encoded               0
Risk_Frequency_Encoded                    0
Triggering Factor_Frequency_Encoded       0
Geoscientific Causes_Frequency_Encoded    0
dtype: int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64


In [5]:
X_numerical

,Longitude,Latitude,Length,Width,Height,LocationCluster,State_Frequency_Encoded,District_Frequency_Encoded,Material Type_Frequency_Encoded,Movement Type_Frequency_Encoded,Failure Plane_Frequency_Encoded,Activity_Frequency_Encoded,Failure Mechanism_Frequency_Encoded,Geomorphology_Frequency_Encoded,Hydrology_Frequency_Encoded,Risk_Frequency_Encoded,Triggering Factor_Frequency_Encoded,Geoscientific Causes_Frequency_Encoded
0,75.778490,33.252910,100,25,80.0,2,0.527108,0.319277,0.087349,0.140060,0.424699,0.073795,0.862952,0.158133,0.243976,0.612952,0.203313,0.004518
1,75.776130,33.253610,100,110,60.0,2,0.527108,0.319277,0.087349,0.140060,0.424699,0.073795,0.862952,0.158133,0.243976,0.612952,0.203313,0.004518
2,75.777560,33.251960,110,35,90.0,2,0.527108,0.319277,0.087349,0.140060,0.424699,0.073795,0.862952,0.158133,0.243976,0.612952,0.203313,0.004518
3,75.776320,33.259960,90,50,80.0,2,0.527108,0.319277,0.117470,0.140060,0.424699,0.073795,0.862952,0.158133,0.243976,0.612952,0.203313,0.001506
4,75.769300,33.275940,60,50,50.0,2,0.527108,0.319277,0.087349,0.140060,0.424699,0.073795,0.862952,0.158133,0.243976,0.612952,0.203313,0.001506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,79.287889,29.697194,25,30,24.0,4,0.472892,0.082831,0.322289,0.769578,0.114458,0.799699,0.114458,0.021084,0.382530,0.105422,0.067771,0.004518
660,79.265528,29.716528,25,30,25.0,4,0.472892,0.082831,0.245482,0.769578,0.114458,0.799699,0.114458,0.194277,0.251506,0.612952,0.067771,0.004518
661,79.141339,29.561249,20,15,18.0,4,0.472892,0.082831,0.322289,0.769578,0.114458,0.799699,0.114458,0.021084,0.251506,0.045181,0.067771,0.034639
662,79.151775,29.565761,60,25,55.0,4,0.472892,0.082831,0.322289,0.769578,0.114458,0.799699,0.114458,0.194277,0.382530,0.612952,0.067771,0.034639
